<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/haar_cascade_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# image processing
import cv2

In [2]:
# download prebuilt haarcascade xml for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_xml/haarcascade_frontalface_default.xml')

# webcam
cap = cv2.VideoCapture(0)

while 1:
  ret, frame = cap.read()
  # haar cascade needs grayscale image so covert bgr to gray (Opencv uses bgr format)
  gray_scale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  # detectMultiScale returns coords of bounding box(s)
  facesRect = face_cascade.detectMultiScale(gray_scale, scaleFactor=1.4, minNeighbors=1, minSize=(30,30))       #1.3 - scalefactor
 
  for (x, y, w, h) in facesRect:
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
    # roi - region of interest 
    roi_gray = gray_scale[y:y + h, x:x + w]
    roi_color = frame[y:y + h, x:x + w]
    
    cv2.imshow('frame', frame)

    # return if esc pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

# release webcam
cap.release()

# close window
cv2.destroyAllWindows()